In [17]:
import numpy as np
import sympy as sp
import math
import scipy.integrate as si
import scipy.optimize as so
import os

In [18]:
# PARAMETERS

concentration = 10 # choose frm the following values: 10, 30, 100
save_to_disk = False # If set to 'True', 'E_fracs' and 'survival fractions' will be saved to disk. Set to 'False' if you don't want them to be saved to disk.

K2021 Method

In [19]:
def second_derivative_parametric_input(x, y, t):
    # Input should be 'x' in terms 't', and 'y' in terms of 't'
    # Output is second derivative of 'y' with respect to 'x' ('d2y/dx2') as a function of 'x'
    dy_dx = sp.Derivative(y, t) / sp.Derivative(x, t)
    d2y_dx2 =  sp.Derivative(dy_dx, t) / sp.Derivative(x, t)
    d2y_dx2_in_terms_of_t = sp.nsimplify(d2y_dx2.doit()) # Computes the second derivative of 'y' with respect to 'x' as a function of 't'

    global psi_Prime
    psi_Prime = sp.symbols("psi_Prime", positive=True)
    eqn = sp.Eq(psi_Prime, x)
    # Python gives the solution in terms of the 0th (principal) branch of the LambertW function. This is the wrong solution.
    sol_t = sp.solve(eqn, t, rational=False)[0]
    
    # Asks python instead to convert the 0th branch of the LambertW function in 'sol_t' to the -1th branch of the LambertW function
    # So, in 'sol_t' the following code converts LambertW(args) to LambertW(args, -1)
    mod_sol_t = sol_t.replace(lambda m: isinstance(m, sp.LambertW), lambda m: sp.LambertW(*m.args, -1))

    # d2ydx2 in terms of psi
    d2y_dx2_in_terms_of_x = sp.nsimplify(d2y_dx2_in_terms_of_t.subs(t, mod_sol_t))


    return d2y_dx2_in_terms_of_x
    

In [20]:
c, f_NFW, x = sp.symbols("c f_NFW x", positive=True)
crho = 1 / (c*x) / (1+c*x)**2 # dimensionless density of NFW profile. 'chro = rho / rho_s', where 'rho' is the density profile and 'rho_s' is the scale radius.
psi = 1 / f_NFW * sp.log(1+c*x) / x # normalized relative potential

d2crho_dpsi2 = second_derivative_parametric_input(psi, crho, x) # Second derivative of 'chro' with respect to 'psi'

In [21]:
c_val = concentration
f_NFW_val = math.log(1+c_val)-c_val/(1+c_val)

alpha_squared = 3/c_val**2 + 1/(2*f_NFW_val)*(c_val-3)/(c_val+1)
beta = (c_val**3 - 2*c_val*(1+c_val)*f_NFW_val) / (2*(1+c_val)**2 * f_NFW_val**2) # this is actually gamma in our article

d2crho_dpsi2_c_and_fNFW_substituted = d2crho_dpsi2.subs({c:c_val, f_NFW:f_NFW_val}) # substitute the numerical values of 'concentration' and 'f_NFW' in 'd2crho_dpsi2'

d2crho_dpsi2_c_and_fNFW_substituted_lambdified = sp.lambdify(psi_Prime, d2crho_dpsi2_c_and_fNFW_substituted) # Lambdify 'd2crho_dpsi2_c_and_fNFW_substituted' so that you can evaluate it by passing a numerical value of 
                                                                                                             # 'psi_Prime' as input


def get_x_c():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - 1/f_NFW_val * np.log(1+c_val*x_c) / x_c
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Compute the normalized crossover radius x*
def integrand(psi_prime, eps, x):
    psi = 1/f_NFW_val * math.log(1+c_val*x) / x

    return (
        x**2 * 1/(math.sqrt(8)*math.pi**2) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified(psi_prime).real * np.sqrt(2*(psi - eps))
    )



# define upper and lower limits of the integrals in 'integral_1'
def range_x_1():
    return [0, min(x_c, 1)]

def range_eps_1(x):
    return [0, beta / alpha_squared * E_frac * x**2]

def range_psi_prime_1(eps, x):
    return [0, eps]



# define upper and lower limits of the integrals in 'integral_2'
def range_x_2():
    return [min(x_c, 1), 1]

def range_eps_2(x):
    psi = 1/f_NFW_val * math.log(1+c_val*x) / x
    return [0, psi]

def range_psi_prime_2(eps, x):
    return [0, eps]


# Load numerical simulation data points: provided to us courtesy of Jacob Shen from the S2023 paper. Load the correct file based on the value of 'concentration'.
if concentration == 10:
    data_SF_vs_Efrac = np.genfromtxt(os.path.join('..', 'Data', 'S2023 Fig 3, c=10.dat'), dtype=None, delimiter='')
elif concentration == 30:
    data_SF_vs_Efrac = np.genfromtxt(os.path.join('..', 'Data', 'S2023 Fig 3, c=30.dat'), dtype=None, delimiter='')
elif concentration == 100:
    data_SF_vs_Efrac = np.genfromtxt(os.path.join('..', 'Data', 'S2023 Fig 3, c=100.dat'), dtype=None, delimiter='')
else:
    raise ValueError("Please set the 'concentration' parameter to one of the following values: 10, 30, 100")


E_fracs = data_SF_vs_Efrac[:,0] # these are the values of 'E_frac' for all the simulation data points from S2023 paper
survival_fractions = np.zeros(np.size(E_fracs)) # Create a zeros array the same size as the 'E_fracs' array. This will eventually contain the values of survival fractions corresponding to E_frac values.

for i in range(np.size(E_fracs)):

    E_frac = E_fracs[i]

    x_c = get_x_c() # compute the normalized crossover radius x*

    # Start computing the survival fraction using triple nested integrals

    integral_1 = si.nquad(integrand, [range_psi_prime_1, range_eps_1, range_x_1])[0]

    if min(x_c, 1) == 1:
        integral_2 = 0
    else:
        integral_2 = si.nquad(integrand, [range_psi_prime_2, range_eps_2, range_x_2])[0]


    survival_fraction = 1 - 4*math.pi*c_val**3/f_NFW_val * (integral_1 + integral_2)  # From eqn (D1) in our article

    survival_fractions[i] = survival_fraction

    print(f"Iteration #{i+1}")



Iteration #1
Iteration #2
Iteration #3
Iteration #4
Iteration #5
Iteration #6


In [22]:
if save_to_disk:
    np.save(f"survival_fractions_K2021_c_{c_val}_ErrorChecking", survival_fractions)

S2023 Fitting Functions 

In [23]:
# Compute the survival fraction using S2023 fitting functions.
eta = 0.98
a1 = -0.8
a2 = -0.59
a3 = -0.034
b0 = -0.58
b1 = -0.56

p = 10**( a1*(np.log10(c_val) - eta) + a2*(np.log10(c_val) - eta)**2 + a3*(np.log10(c_val) - eta)**3 )
k = 10**( b0 + b1*(np.log10(c_val) - 2) )

survival_fractions_S2023 = 2 / ( 1+ (1 + E_fracs/p)**k )

In [24]:
if save_to_disk:
    np.save(f"survival_fractions_S2023_c_{c_val}_ErrorChecking", survival_fractions_S2023)